In [ ]:
# %% Deep learning - Section 17.160
#    Code challenge 25: run an experiment on a GPU
#
#    1) Start from code from video 11.111 (mnist dataset)
#    2) Train the model on the GPU
#    3) Keep track of the time on GPU (it was ~14 mins on CPU)

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [1]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Note

# To run models on a GPU you must select from the menu:
#   -> Runtime
#     -> Change runtime type
#       -> Hardware accelerator
#         -> GPU


In [ ]:
# %% Use GPU

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


In [2]:
# %% Data

# Load data
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# Split labels from data
labels = data[:,0]
data   = data[:,1:]

# Normalise data (original range is (0,255))
data_norm = data / np.max(data)


In [3]:
# %% Create train and test datasets

# Convert to tensor (float and integers)
data_tensor   = torch.tensor(data_norm).float()
labels_tensor = torch.tensor(labels).long()

# Split data with scikitlearn (10% test data)
train_data,test_data,train_labels,test_labels = train_test_split(data_tensor,labels_tensor,test_size=0.1)

# Convert to PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [5]:
# %% Function to generate the model
#    Flexibly loop over model depth/breadth

def gen_model(nUnits,nLayers,drop_rate):

    class mnist_FFN(nn.Module):
        def __init__(self,nUnits,nLayers,dropout_rate):
            super().__init__()

            # Dictionary to store the layers
            self.layers  = nn.ModuleDict()
            self.nLayers = nLayers

            # Dropout
            self.dropout_rate = dropout_rate

            # Architecture
            self.layers['input'] = nn.Linear(784,nUnits)
            for i in range(nLayers):
                self.layers[f'hidden{i}'] = nn.Linear(nUnits,nUnits)
            self.layers['output'] = nn.Linear(nUnits,10)

        # Forward propagation
        def forward(self,x):

            x = F.relu(self.layers['input'](x))
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

            for i in range(self.nLayers):
                x = F.relu(self.layers[f'hidden{i}'](x))
                x = F.dropout(x, p=self.dropout_rate, training=self.training)

            x = self.layers['output'](x)

            return x

    # Create model instance
    ANN = mnist_FFN(nUnits,nLayers,drop_rate)

    # Loss function
    loss_fun = nn.CrossEntropyLoss()

    # Optimizer (SGD to slow down learning for illustration purpose)
    optimizer = torch.optim.SGD(ANN.parameters(),lr=0.01)

    return ANN,loss_fun,optimizer


In [12]:
# %% Function to train the model

def train_model(nUnits,nLayers,drop_rate,device=device):

    # Parameters, model instance, inizialise vars
    num_epochs = 60
    ANN,loss_fun,optimizer = gen_model(nUnits,nLayers,drop_rate)

    # Ship model to GPU
    ANN.to(device)

    losses    = []
    train_acc = []
    test_acc  = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Ship data to GPU
            X = X.to(device)
            y = y.to(device)

            # Forward propagation and loss
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch (ship data back)
            batch_loss.append(loss.item())

            yHat = yHat.cpu()
            y    = y.cpu()

            matches     = torch.argmax(yHat,axis=1) == y
            matches_num = matches.float()
            accuracy    = 100 * torch.mean(matches_num)
            batch_acc.append(accuracy)

        losses.append( np.mean(batch_loss) )
        train_acc.append( np.mean(batch_acc) )

        # Test accuracy
        ANN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))

            # Ship to GPU (X only, y is only for accuracy)
            X = X.to(device)

            yHat = ANN(X)

            # Ship back
            yHat = yHat.cpu()

            test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

        ANN.train()

    return train_acc,test_acc,losses,ANN


In [13]:
# %% Parametric experiment on model depth and breadth
#    This cell takes ~ 14 mins on CPU and ~18 mins on CPU

# Define model parameters (num of hidden layers and units per hidden layer)
num_layers = np.linspace(1,3,3,dtype=int)
num_units  = np.linspace(50,250,5,dtype=int)
drop_rate  = 0.25

# Preallocate output matrices
accuracies_train = np.zeros(( len(num_units),len(num_layers) ))
accuracies_test  = np.zeros(( len(num_units),len(num_layers) ))
training_times   = np.zeros(( len(num_units),len(num_layers) ))

# Buckle up, here's the experiment!
for unit_i in range(len(num_units)):
    for layer_i in range(len(num_layers)):

        start_time = time.time()

        # Run model and store outputs
        train_acc,test_acc,losses,ANN = train_model(num_units[unit_i],num_layers[layer_i],drop_rate)

        duration = time.time() - start_time

        # Store accuracies as average accuracy over last 5 epochs, and time
        accuracies_train[unit_i,layer_i] = np.mean(train_acc[-5:])
        accuracies_test[unit_i,layer_i]  = np.mean(test_acc[-5:])
        training_times[unit_i,layer_i]   = duration


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig, axs = plt.subplots(1,2,figsize=(1.5*6*phi,6))

cmaps = plt.cm.plasma(np.linspace(.2,.9,len(num_layers)))

# Plot for Training Accuracy
for i, nl in enumerate(num_layers):
    axs[0].plot(num_units,accuracies_train[:, i],'-o',color=cmaps[i],label=f'{int(nl)} layers')
axs[0].set_title('Training accuracy')
axs[0].set_xlabel('Number of units')
axs[0].set_ylabel('Accuracy (%)')
axs[0].legend(title='Hidden layers')
axs[0].grid(True)
axs[0].set_ylim(94.8,100.2)

# Plot for Test Accuracy
for i, nl in enumerate(num_layers):
    axs[1].plot(num_units,accuracies_test[:,i],'-o',color=cmaps[i],label=f'{int(nl)} layers')
axs[1].set_title('Test accuracy')
axs[1].set_xlabel('Number of units')
axs[1].set_ylabel('Accuracy (%)')
axs[1].legend(title='Hidden layers')
axs[1].grid(True)
axs[1].set_ylim(94.8,100.2)

plt.suptitle('Model performance vs. width and depth', fontsize=14)
plt.tight_layout()

plt.savefig('figure2_code_challenge_25.png')
plt.show()
files.download('figure2_code_challenge_25.png')


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig, axs = plt.subplots(1,2,figsize=(1.5*6*phi,6))

cmaps = plt.cm.plasma(np.linspace(.2,.9,len(num_units)))

# Plot for Training Accuracy
for i,n_units in enumerate(num_units):
    axs[0].plot(num_layers,accuracies_train[i,:],'-o',color=cmaps[i],label=f'{int(n_units)} units')
axs[0].set_title('Training accuracy')
axs[0].set_xlabel('Number of hidden layers')
axs[0].set_ylabel('Accuracy (%)')
axs[0].legend(title='Units per layer')
axs[0].grid(True)
axs[0].set_ylim(94.8,100.2)

# Plot for Test Accuracy
for i,n_units in enumerate(num_units):
    axs[1].plot(num_layers,accuracies_test[i,:],'-o',color=cmaps[i],label=f'{int(n_units)} units')
axs[1].set_title('Test accuracy')
axs[1].set_xlabel('Number of hidden layers')
axs[1].set_ylabel('Accuracy (%)')
axs[1].legend(title='Units per layer')
axs[1].grid(True)
axs[1].set_ylim(94.8,100.2)

plt.suptitle('Model performance vs. breadth and depth',fontsize=14)
plt.tight_layout()

plt.savefig('figure3_code_challenge_25.png')
plt.show()
files.download('figure3_code_challenge_25.png')


In [ ]:
# %% Plotting

#Plot time matrix
fig, ax = plt.subplots(figsize=(phi*6,6))
im = ax.imshow(training_times,cmap='plasma',aspect='auto')

ax.set_xticks(range(len(num_layers)))
ax.set_xticklabels([int(n) for n in num_layers])
ax.set_xlabel('Number of Layers')

ax.set_yticks(range(len(num_units)))
ax.set_yticklabels([int(n) for n in num_units])
ax.set_ylabel('Number of Units')

cbar = plt.colorbar(im,ax=ax)
cbar.set_label('Training time (s)',rotation=270,labelpad=15)

threshold = (training_times.max() + training_times.min()) / 2

for i in range(len(num_units)):
    for j in range(len(num_layers)):
        value = training_times[i,j]
        ax.text(j,i,f'{value:.0f} s',ha='center',va='center',color='white' if value < threshold else 'black')

total_secs = np.sum(training_times)
mins       = int(total_secs // 60)
secs       = int(total_secs % 60)
ax.set_title(f'Training time per model configuration (GPU)\n(Total time = {mins}:{secs} mins)')

plt.tight_layout()

plt.savefig('figure4_code_challenge_25.png')
plt.show()
files.download('figure4_code_challenge_25.png')
